In [ ]:
import json

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import numpy as np
import pandas as pd
from tqdm import tqdm

import lightgbm as lgbm

In [ ]:
SUBMISSION_RUN = False

In [ ]:
# Constants

# Paths
INPUT_BASE_PATH = "/kaggle/input/"
RAW_DATA_INPUT_PATH = f"{INPUT_BASE_PATH}/m5-forecasting-accuracy"
PROCESSED_DATA_INPUT_PATH = f"{INPUT_BASE_PATH}/m5-acc"
OUTPUT_BASE_BATH = "/kaggle/working"

# Timestamps
MAX_TRAIN_TIMESTAMP = 1941
START_TEST_TIMESTAMP = 1942
START_TEST_WM_YR_WK = 11617

# Data Input

In [ ]:
# Load raw data

# CALENDAR_DATA = pd.read_csv(f"{RAW_DATA_INPUT_PATH}/calendar.csv")
# SELL_PRICES = pd.read_csv(f"{RAW_DATA_INPUT_PATH}/sell_prices.csv")
# SALES_TRAIN_EVALUATION = pd.read_csv(f"{RAW_DATA_INPUT_PATH}/sales_train_evaluation.csv")

SAMPLE_SUBMISSION = pd.read_csv(f"{RAW_DATA_INPUT_PATH}/sample_submission.csv")
SUBMISSION_INDEX = SAMPLE_SUBMISSION.set_index("id").index
VAL_SUBMISSION = SAMPLE_SUBMISSION[SAMPLE_SUBMISSION["id"].str.contains("validation")]
EVAL_SUBMISSION = SAMPLE_SUBMISSION[SAMPLE_SUBMISSION["id"].str.contains("evaluation")]

In [ ]:
# Load preprocessed data
train_data_set = pd.read_parquet(f"{PROCESSED_DATA_INPUT_PATH}/m5-acc-train.parquet")

In [ ]:
def cast_data_types(dataset: pd.DataFrame) -> pd.DataFrame:
    item_category_cols = ["dept_id", "cat_id", "store_id", "state_id"]
    date_category_cols = ["wm_yr_wk", "weekday", "month", "year", "event_name", "event_type", "snap_CA", "snap_TX", "snap_WI"]
    for category_col in item_category_cols + date_category_cols:
        try:
            dataset[category_col] = dataset[category_col].astype("category")
        except KeyError:
            print(f"Column {col} does not exist. Skipping ...")
            pass
    
    col_to_numeric_dtype = [
        ("item_id", "float"),
        ("d", "integer"),
        ("count", "integer"),
        ("sell_price", "float"),
        ("weekday_sin", "float"),
        ("weekday_cos", "float"),
        ("month_sin", "float"),
        ("month_cos", "float"),
        ("av_item_state_sell_price", "float"),
        ("av_dept_state_sell_price", "float"),
    ]
    for (col, dtype) in col_to_numeric_dtype:
        try:
            dataset[col] = pd.to_numeric(dataset[col], downcast=dtype)
        except KeyError:
            print(f"Column {col} does not exist. Skipping ...")
            pass
    
    return dataset

In [ ]:
# Cast to lower resolution types to save memory
train_data_set = cast_data_types(train_data_set)

# LightGBM Model

In [ ]:
# Rolling window validation

def time_series_split(
    df: pd.DataFrame,
    n_folds: int = 5,
    horizon: int = 28,
    overlap: int = 0,
    max_timestamp: int = MAX_TRAIN_TIMESTAMP,
):
    min_timestamp = max_timestamp - n_folds * horizon + (n_folds - 1) * overlap
    for fold_idx in range(n_folds):
        start = min_timestamp + fold_idx * (horizon - overlap)
        stop = start + horizon

        train_data = df[df["d"] < start].copy()
        valid_data = df[(df["d"] >= start) & (df["d"] < stop)].copy()
        
        print(f"Fold index: {fold_idx}")
        print(f"Train idx (start, end): ({train_data['d'].min()}, {train_data['d'].max()})")
        print(f"Valid idx (start, end): ({valid_data['d'].min()}, {valid_data['d'].max()})")
        print("==================")
        
        yield train_data, valid_data
        
        
def train_predict_score(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    X_test: pd.DataFrame,
    y_test: pd.Series,
) -> tuple[float, float]:
    train_data = lgbm.Dataset(data=X_train, label=y_train)
    valid_data = lgbm.Dataset(data=X_test, label=y_test)
    model = lgbm.train(
        PARAMETERS,
        num_boost_round=1000,
        train_set=train_data,
        valid_sets=[valid_data],
        callbacks=[
            lgbm.early_stopping(stopping_rounds=5),
            lgbm.log_evaluation(period=10),
        ]
    )
    
    y_hat = model.predict(X_test)
    y_hat = np.clip(np.round(y_hat), a_min=0, a_max=np.inf)
    y_hat = y_hat.astype(np.int32)
    rmse = np.sqrt(np.mean((y_test - y_hat) ** 2))

    return rmse, model.best_iteration

In [ ]:
FEATURES = [
    "item_id",
    "dept_id",
    "cat_id",
    "store_id",
    "state_id",
    "weekday_sin",
    "weekday_cos",
    "month_sin",
    "month_cos",
    "year",
    "event_name",
    "event_type",
    "snap_CA",
    "snap_TX",
    "snap_WI",
    "sell_price",
    "av_item_state_sell_price",
    "av_dept_state_sell_price",
]
LABEL = "count"

PARAMETERS = {
    "objective": "poisson",
    "learning_rate": 0.05,
    "metric": "rmse",
    "force_col_wise": True,
}

In [ ]:
if not SUBMISSION_RUN:
    scores = []
    best_iterations = []
    
    for train, valid in tqdm(time_series_split(data, n_folds=5), total=5):
    
        X_train, y_train = train[FEATURES], train[LABEL]
        X_test, y_test = valid[FEATURES], valid[LABEL]
        score, best_iter = train_predict_score(X_train, y_train, X_test, y_test)
        scores.append(score)
        best_iterations.append(best_iter)
    
    cv_results = {
        "scores": scores,
        "mean_score": np.mean(scores),
        "std_score": np.std(scores),
        "best_iterations": best_iterations,
        "mean_best_iteration": np.mean(best_iterations)
    }
    
    with open(f"{OUTPUT_BASE_BATH}/cv_results.json", "w") as f:
        json.dump(cv_results, f)
    
else:
    train_data = lgbm.Dataset(data=data[FEATURES], label=data[LABEL])
    model = lgbm.train(
        PARAMETERS,
        num_boost_round=400,
        train_set=train_data,
    )

# Forecast & Submit

In [ ]:
def construct_test_df(
    d_start: int,
    d_end: int,
    item_category_df: pd.DataFrame,
    calendar_df: pd.DataFrame,
    price_df: pd.DataFrame,
) -> pd.DataFrame:
    
    d_range = list(range(d_start, d_end + 1))
    d_range_df = pd.DataFrame(d_range, columns=["d"])
    test_df = pd.merge(item_id_categories, d_range_df, how="cross")
    test_df = merge_sales_and_calendar_data(test_df, calendar_df)
    test_df = merge_sales_and_price_data(test_df, price_df)
    return test_df

def map_test_df_columns(test_df: pd.DataFrame) -> pd.DataFrame:
    test_df["item_id"] = test_df["item_id"].map(item_id_map)
    test_df["dept_id"] = test_df["dept_id"].map(dept_id_map)
    test_df["cat_id"] = test_df["cat_id"].map(cat_id_map)
    test_df["store_id"] = test_df["store_id"].map(store_id_map)
    test_df["state_id"] = test_df["state_id"].map(state_id_map)
    test_df["event_name"] = test_df["event_name"].map(event_name_map)
    test_df["event_type"] = test_df["event_type"].map(event_type_map)
    return test_df

def pivot_forecast_df(X_test: pd.DataFrame, y_hat: np.ndarray) -> pd.DataFrame:
    forecast_df = pd.concat([X_test, pd.DataFrame(y_hat, columns=["count"])], axis=1)
    forecast_df = forecast_df.pivot(columns="d", index="id", values="count").sort_index(axis=1)
    forecast_df = forecast_df.rename_axis(None, axis=1).reset_index()
    return forecast_df

def rename_forecast_columns(forecast_df: pd.DataFrame, forecast_horizon: int = 28) -> pd.DataFrame:
    forecast_df = forecast_df.set_index("id")
    forecast_df.columns = [f"F{i}" for i in range(1, forecast_horizon + 1)]
    return forecast_df.reset_index()

In [ ]:
if SUBMISSION_RUN:
    test_d_start, test_d_end = 1942, 1969
    test_df = construct_test_df(
        d_start=test_d_start,
        d_end=test_d_end,
        item_category_df=item_id_categories,
        calendar_df=calendar_df,
        price_df=price_df,
    )
    test_df = map_test_df_columns(test_df)
    test_df = cast_types(test_df)

    y_hat = model.predict(test_df[FEATURES])
    y_hat = np.clip(np.round(y_hat), a_min=0, a_max=np.inf)
    y_hat = y_hat.astype(np.int32)

    forecast_df = pivot_forecast_df(test_df, y_hat)
    forecast_df = rename_forecast_columns(forecast_df)
    
    # Merge with sample submissions
    EVAL_SUBMISSION = EVAL_SUBMISSION[["id"]].merge(forecast_df, on="id", how="left")
    FINAL_SUBMISSIONS = pd.concat([VAL_SUBMISSION, EVAL_SUBMISSION])
    FINAL_SUBMISSIONS = FINAL_SUBMISSIONS.set_index("id").reindex(SUBMISSION_INDEX).reset_index()
    
    FINAL_SUBMISSIONS.to_csv(f"{OUTPUT_BASE_BATH}/submission.csv", index=False)
    